In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime
import random


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int

In [2]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


# Existing Quantization schemes

In [3]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")



e:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.10.4+0.x64.w64.mingw32. You currently have `1.10.2+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.4+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


QSGD
Before Quantization:  [0.49434513 6.2857423  1.3571445  0.16863571 3.187139   5.421778
 8.057935   5.2366385  5.9929357  3.2144673  9.129675   8.294142
 5.387526   3.3563554  8.107922   4.8219156 ]
Encoded b'\'\x00\x00\x00\xbc\x19\xb3Ae\x91"\x93\x02\x00\x00\x00'
After Quantization:  [ 0.       11.193783  0.        0.        0.        0.       11.193783
  0.       11.193783 11.193783  0.        0.       11.193783  0.
 11.193783  0.      ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [4]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


GZip
Before Quantization:  [5.2759366  3.462241   8.660746   6.233553   1.0147341  2.4925554
 7.5946326  5.0407968  7.4687757  4.5906806  6.3286467  6.54555
 0.48748773 2.8149178  2.9470851  8.557952  ]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\ncd\x00\x01F0\x02\x81\xba\x13\x1b\x1d\x01\xd5`j\xde\x14\x00\x00\x00'
After Quantization:  [22.222897  0.        0.        0.        0.        0.       22.222897
  0.        0.       22.222897  0.        0.        0.        0.
  0.        0.      ]
38.46153846153846% smaller 
61.904761904761905% smaller 


In [5]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


FP8
Before Quantization:  [9.767468   8.524345   8.906465   2.9491475  2.9696865  3.4505231
 3.152149   2.8923426  7.176006   5.157509   8.652066   2.2861204
 4.565732   3.617773   8.163802   0.50705504]
Encoded b'IHHBBCCAGFH@ECH8'
After Quantization:  [10.   8.   8.   3.   3.   3.5  3.5  2.5  7.   6.   8.   2.   5.   3.5
  8.   0.5]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [6]:

import cv2


class customCompression():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    



#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint8
cmp = customCompression(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[9.976559   9.058106   5.528075   3.6162739  5.591813   1.0089798
 8.144225   5.179376   6.47697    6.1146693  4.0610228  6.394287
 4.3792863  4.6727476  5.0939302  0.20706782]
Encoded [255 231 138  88 140  20 207 129 163 154 100 161 108 116 127   0]
After Quantization:  [9.97655869 9.05707719 5.4940864  3.57849996 5.57070986 0.9733024
 8.1375957  5.14928084 6.45187963 6.10707407 4.03824071 6.37525617
 4.34473454 4.65122837 5.07265739 0.20706782]
<class 'numpy.uint8'> 48.275862068965516% smaller 
<class 'numpy.uint8'> 28.571428571428573% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  0.0005580578098404815


In [7]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))
print(sys.getsizeof(enc))

168
232
120


# Custom Frequency Domain Quantization

In [8]:
from scipy.fftpack import dct, idct

class customFreq_1_point_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_point_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.0496483  0.8561303  0.57366997 0.23970783 0.57884187 0.37662703
 0.5351799  0.2931987  0.9321833  0.4845873  0.8469219  0.9358102
 0.86745584 0.52255267 0.6412494  0.21764001]
Encoded [ 0.105    0.714    1.132    0.3604   0.7847   0.56     0.744    0.4622
  0.998    0.603    0.696    0.7036   0.322   -0.04208  0.02272 -0.2793 ]
After Quantization:  [0.04960848 0.8559898  0.5736593  0.23961355 0.5789125  0.3766538
 0.5352592  0.29310697 0.93217367 0.48464674 0.84682566 0.93564284
 0.8673148  0.522491   0.6412308  0.21747984]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  8.83295e-09


In [9]:
from scipy.fftpack import dct, idct

class customFreq_singleDCT():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_singleDCT()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.5272018  0.44428557 0.13158825 0.43073168 0.7563973  0.12540302
 0.71841717 0.92661005 0.58108944 0.1265346  0.5155451  0.3572595
 0.97099495 0.6294249  0.5304033  0.8573565 ]
Encoded [ 2.158   -0.351    0.03888 -0.2427   0.1796   0.3503  -0.1433  -0.0878
  0.5464   0.125    0.02379 -0.5205  -0.08545  0.0201  -0.0986   0.345  ]
After Quantization:  [0.5273747  0.44442576 0.13182306 0.43090317 0.7565172  0.12577814
 0.7186809  0.9267225  0.58130336 0.12678543 0.51572764 0.35758308
 0.9712175  0.62966263 0.5307491  0.8575577 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  5.5259967e-08


# Custom Freq 3 (CustomFreq + Custom2 but more sophisticated)

In [10]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)
        return enc
    
    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_3():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [np.uint32, np.uint16, np.uint8]):
        self.precision_levels = precision_levels
        self.linearQuant_0 = Linear_quant(self.precision_levels[0])
        self.linearQuant_1 = Linear_quant(self.precision_levels[1])
        self.linearQuant_2 = Linear_quant(self.precision_levels[2])
        
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        self.mean_importance = np.mean(importance)
        # if self.mean_importance > 0.1:
        enc = self.linearQuant_0.encode(weights_f)  # 8-bit
        precision = int(str(self.linearQuant_0.quantization_type).split('t')[-1].split('\'')[0])
        print(precision)

        # elif self.mean_importance > 0.01:
        #     enc = self.linearQuant_1.encode(weights_f)   # 4-bit
        #     precision = int(str(self.linearQuant_1.quantization_type).split('t')[-1].split('\'')[0])
        #     # enc = self.quantize(weights_f, 16)
        # else:
        #     enc = self.linearQuant_2.encode(weights_f)   # 2-bit
        #     precision = int(str(self.linearQuant_2.quantization_type).split('t')[-1].split('\'')[0])

        
        # Quantize frequency components
        # enc = self.quantize(enc, precision)
        return enc


    def decode(self, enc) :

        # if self.mean_importance > 0.1:
        weights_quantized = self.linearQuant_0.decode(enc)  # 8-bit
        # elif self.mean_importance > 0.01:
        #     weights_quantized = self.linearQuant_1.decode(enc)   # 4-bit
        # else:
        #     weights_quantized = self.linearQuant_2.decode(enc)   # 2-bit
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_3()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.3894869  0.22704653 0.81469357 0.8928578  0.50958544 0.7276715
 0.06012404 0.10388535 0.45995986 0.35798022 0.9554318  0.5902813
 0.961132   0.9552744  0.9239553  0.6108902 ]
32
Encoded [1975125186 1874146273 2880414898 4294967295 2614741622 2780607574
 1469870690  702649080 2798709260 2104753531 3761357890 2199753246
 2696075855 2009507680 1021178275          0]
After Quantization:  [0.38948683 0.22704645 0.81469342 0.89285772 0.50958533 0.72767147
 0.06012401 0.10388536 0.45995985 0.35798022 0.95543174 0.5902813
 0.96113189 0.9552743  0.92395529 0.6108902 ]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  5.198560856392657e-15


# CustomFreq2.5

In [11]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')
    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')
    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_point_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.6912875  0.0504155  0.57657236 0.3055623  0.30354652 0.967246
 0.5597237  0.19770943 0.8777293  0.54320484 0.39535806 0.25646353
 0.07100581 0.38083583 0.77854604 0.01752366]
Encoded [ 0.8037   0.3513   0.525    0.672    0.3533   0.959    0.7935   0.11914
  0.759    0.409    0.06915  0.02812 -0.06647  0.28     0.3557  -0.4326 ]
After Quantization:  [0.6912881  0.0504888  0.5766083  0.30544809 0.30364695 0.9669671
 0.5597018  0.19764775 0.8777512  0.54307675 0.39536753 0.25642776
 0.07103936 0.3806445  0.77856296 0.0176161 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.1042252e-08


# Custom Freq 2 (CustomFreq + Custom2) 
- best version

In [12]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_2()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.06627755 0.699335   0.5618062  0.73616964 0.1570748  0.7372298
 0.7915822  0.21583925 0.5715141  0.9876023  0.68774587 0.47195846
 0.5177013  0.7360376  0.84565836 0.57027096]
Encoded [ 361975257 3329856653 3335412284 3886328392 2220852895 2811923852
 4294967295 1565482326 2557717003 3975832711 2242583092 1134967289
 1541281541 1937165260 1451891396          0]
After Quantization:  [0.0662775  0.69933493 0.56180615 0.73616957 0.15707469 0.73722986
 0.79158219 0.21583921 0.57151407 0.98760212 0.68774576 0.47195844
 0.51770118 0.73603766 0.84565825 0.57027096]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  6.809387884718506e-15


# Selected Schemes

In [13]:
class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")
Main.include("src/compressors/fp8.jl")

number_of_selected_schemes = 4
column_names = ["accuracy","time","efficiency"]
metrics_cell = np.zeros((number_of_selected_schemes, len(column_names)), dtype=object)

# list = []
# list.append(dict)
# dict[qs]={{a,t,e}





#______________QSGD________________
class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle
        

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)

# cmp = GZip(1)

#______________FP8________________
class FP8(Compressor):
    def __init__(self):
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
# cmp = FP8()




#______________Custom Linear________________
class Custom_linear():
    def __init__(self, quantization_type = np.uint32):
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        
    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec

# data_type = np.uint8
# cmp = Custom_linear(data_type)

#______________Frequency Domain Quantization________________



class customFreq_2_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        self.precision_levels = precision_levels
        self.linearQuant = Custom_linear()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')
    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')
    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
# cmp = customFreq_2_5()





In [ ]:
def get_time_efficiency(cmp):
    


In [15]:
d = {} 
# with open("dict.txt") as f:
#     for line in f:
#         (key, val) = line.split("-")
#         dict = {}
#         i = val.split("|")
#         for j in i:
#             (k, v) = j.split(",")
#             dict[k] = int(v)
#         d[key] = dict

d['QSGD'] = {'acc':1 , 'time': 1, 'compresssion' :3 }


{'QSGD': {'acc': 1, 'time': 1, 'compresssion': 3}}

In [ ]:
d['QSGD']['compression']

In [20]:
d = {'scheme_name': 'QSGD','acc':1 , 'time': 1, 'compresssion' :3 }

df = pd.DataFrame([d])
df

,scheme_name,acc,time,compresssion
0,QSGD,1,1,3


In [25]:
df.loc[df.scheme_name == 'QSGD', 'time'] = 2
df

,scheme_name,acc,time,compresssion
0,QSGD,1,2,3


In [ ]:
compression = pd.read_excel(r"E:\MS_Thesis\mid-defense-april\Results\qunatization_compression_eff.xlsx")
accuracy = pd.read_excel(r"E:\MS_Thesis\Pre-defense-june\Results_for_update\Results_ACC_LOSS.xlsx")

# accuracy = 

# Federated Implementation

In [37]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp_list, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp = None
		self.cmp_list = cmp_list
		self.client_ids = list(range(0,number_of_clients))
		self.bw_traffic = [[random.randint(20, 2500), random.randint(1,50)] for _ in range(number_of_clients)]
#a list of tuple of shape(list) number_of_clients

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        
        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100
		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights




	def get_client_bw_and_traffic(self,client_id):
		bw,traffic = self.bw_traffic[client_id]
		return bw,traffic
	
	def quanization_scheme(self,bw,traffic):
		if bw>15:
			selected_cmp = self.cmp_list[0]
		else:
			selected_cmp = self.cmp_list[1]		
			
		return selected_cmp


	def client_generator(self, train_x, train_y):
		
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()		
		return weights, running_loss, start_size,end_size
	
	
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for selection_index,(dataset_x, dataset_y )in enumerate(clients):
				client_id = selections[selection_index]
				bw,traffic = self.get_client_bw_and_traffic(client_id)
				self.cmp = self.quanization_scheme(bw,traffic)
				# print(self.cmp)
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return agg_weight

#%%

In [38]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customCompression(np.uint8)
cmp_list = [customFreq_2(),FP8()]


# cmp = FP8()
# cmp = QSGD(2,True)
m_8 = MNIST_PAQ_COMP(cmp_list=cmp_list, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
agg_weights = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)

  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_21032\585678726.py:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30115}: 100%|██████████| 164/164 [00:32<00:00,  5.11it/s]


Test Accuracy: 0.09644
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 2.07555}: 100%|██████████| 164/164 [00:32<00:00,  5.05it/s]


Test Accuracy: 0.40457
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 3, 'Loss': 1.85966}: 100%|██████████| 164/164 [00:33<00:00,  4.93it/s]


...

Test Accuracy: 0.92549
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.52937}: 100%|██████████| 164/164 [00:32<00:00,  4.98it/s]


Test Accuracy: 0.93313
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.52629}: 100%|██████████| 164/164 [00:33<00:00,  4.88it/s]


Test Accuracy: 0.93639
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  345.98965525627136
